<a href="https://colab.research.google.com/github/brandontan99/Self_Love_App/blob/master/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!git clone https://github.com/brandontan99/Self_Love_App.git
%cd Self_Love_App
!pip install scikit-multilearn

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifierCV
from skmultilearn.adapt import *
from skmultilearn.problem_transform import *
from skmultilearn.ensemble import * 
from skmultilearn.cluster import *
from sklearn.model_selection import GridSearchCV, KFold
import pandas as pd
from data_cleaning import data_cleaning
from Data_Normalization import data_encoding, data_normalization
import time

Cloning into 'Self_Love_App'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 34 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.
/content/Self_Love_App
     |████████████████████████████████| 92kB 4.9MB/s 


# Data Preparation

In [6]:
df = pd.read_csv("WID3006 ML Questionnaire.csv")
df = data_cleaning(df)
df = data_encoding(df)
df_norm = data_normalization(df)
df_norm

,Do you enjoy socializing with large groups of people?,What is your current occupation?_Employed,What is your current occupation?_Retired,What is your current occupation?_Unemployed,What is your current occupation?_University student,What are your hobbies? (You may select more than 1)_Badminton;Collecting;Exercising,What are your hobbies? (You may select more than 1)_Badminton;Cooking;Reading;Writing,What are your hobbies? (You may select more than 1)_Badminton;Cooking;Writing;Painting,What are your hobbies? (You may select more than 1)_Badminton;Dancing;Playing board games;Collecting,What are your hobbies? (You may select more than 1)_Badminton;Dancing;Reading;Singing;Listening to music,What are your hobbies? (You may select more than 1)_Badminton;Dancing;Watching TV series,What are your hobbies? (You may select more than 1)_Badminton;Exercising,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Cooking,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Cooking;Writing;Singing,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Dancing;Fishing;Gardening;Playing computer games;Playing board games;Painting;Collecting;Puzzles,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Dancing;Playing computer games;Cooking;Reading,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Exercising;Cooking,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Exercising;Dancing;Gardening;Cooking;Reading;Writing;Painting;Collecting;Puzzles;Singing,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Exercising;Playing computer games;Playing board games;Cooking;Reading;Writing;Painting;Collecting;Puzzles,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Exercising;Reading;Singing;Photography and Videography,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Exercising;Singing,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Playing board games;Reading;Singing,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Playing computer games,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Playing computer games;Playing board games;Writing;Collecting;Puzzles,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Playing computer games;Reading;Writing;Painting,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Puzzles,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Reading;Photography and Videography,What are your hobbies? (You may select more than 1)_Badminton;Exercising;Team sports;Sleeping,What are your hobbies? (You may select more than 1)_Badminton;Gardening;Cooking;Reading,What are your hobbies? (You may select more than 1)_Badminton;Gardening;Playing computer games;Reading;Writing;Collecting,What are your hobbies? (You may select more than 1)_Badminton;Gardening;Playing computer games;Reading;Writing;Painting,What are your hobbies? (You may select more than 1)_Badminton;Playing computer games,What are your hobbies? (You may select more than 1)_Badminton;Playing computer games;Cooking;Reading,What are your hobbies? (You may select more than 1)_Badminton;Playing computer games;Cooking;Singing,What are your hobbies? (You may select more than 1)_Badminton;Playing computer games;Playing board games,What are your hobbies? (You may select more than 1)_Badminton;Playing computer games;Playing board games;Cooking;Reading;Painting;Puzzles,What are your hobbies? (You may select more than 1)_Badminton;Playing computer games;Playing board games;Reading,What are your hobbies? (You may select more than 1)_Badminton;Playing computer games;Reading,What are your hobbies? (You may select more than 1)_Badminton;Playing computer games;Singing,What are your hobbies? (You may select more than 1)_Badminton;Reading,...,What is your favorite time of the 

# Models

In [ ]:
# Base Classifiers ( dict format --> (name, func) : params )
base_models = {
    ("Decision Tree", DecisionTreeClassifier) : {'min_samples_split':range(2,5) , 'min_samples_leaf':range(1,4),'min_weight_fraction_leaf':[0.0,0.25,0.5,0.75,1.0],'random_state':range(0,1)},
    ("Extra Tree", ExtraTreeClassifier) : {'min_samples_split':range(2,5) , 'min_samples_leaf':range(1,4),'min_weight_fraction_leaf':[0.0,0.25,0.5,0.75,1.0],'random_state':range(0,1)},
    ("Extra Trees", ExtraTreesClassifier) : {'n_estimators':range(90,120),'min_samples_split':range(2,5) , 'min_samples_leaf':range(1,4),'min_weight_fraction_leaf':[0.0,0.25,0.5,0.75,1.0],'random_state':range(0,1)},
    ("KNN", KNeighborsClassifier) : {'n_neighbors':range(3,8),'leaf_size':range(20,40),'p':range(1,3)},
    ("Radius Neighbours", RadiusNeighborsClassifier) : {'radius':[1.0,1.5,2.0],'leaf_size':range(20,40),'p':range(1,3)},
    ("Random Forest", RandomForestClassifier) :{'n_estimators':range(90,120),'min_samples_split':range(2,5),'min_samples_leaf':range(1,4),'min_weight_fraction_leaf':[0.0,0.25,0.5,0.75,1.0],'random_state':range(0,1)},
    ("MLP", MLPClassifier) : {'alpha':[1e-3, 1e-2, 1e-1, 1],'learning_rate_init':[0.0005,0.001,0.003,0.005,0.01,0.03,0.05],'power_t':[0.2,0.4,0.5,0.6,0.8], 'max_iter':range(200,400),'random_state':range(0,1),'tol':[1e-4,1e-3, 1e-2, 1e-1, 1]},
    ("Ridge" : RidgeClassifierCV) : {'alphas':[1e-3, 1e-2, 1e-1, 1.0,10.0]}
}

# Adaptation approaches
adapt_models = {
    ("Binary Relevance kNN A", BRkNNaClassifier) : {'k': range(1,3)},
    ("Binary Relevance kNN B", BRkNNbClassifier) : {'k': range(1,3)},
    ("Multi-Label KNN", MLkNN) : {'k': range(1,3), 's': [0.5, 0.7, 1.0]},
    ("ARAM Neural Network", MLARAM) : {'threshold':0.05, 'vigilance':0.95},
    ("Twin SVM" : MLTSVM) : {'c_k': [2**i for i in range(-5, 5, 2)]}
}

# Problem Transformation
problem_transform = {
    "Binary Relevance" : BinaryRelevance,
    "Classifier Chain" : ClassifierChain,
    "Label Powerset" : LabelPowerset
}

# Ensembles of Classifiers
emsemble_methods = {
    "Distinct Random k-labtest" : RakelD,
    "Overlapping Random k-labtest" : RakelO,
    "Label Space Partitoning" : LabelSpacePartitioningClassifier,
    "Majoity Voting" : MajorityVotingClassifier
}
# Multi-label embeddings (KIV)

# Hyperparameters

In [ ]:
# Hyperparameters Tuning
#clf = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1)
num_of_kfold_splits = 10
seed = 

# Training

### Adaptation approaches

In [ ]:
results = {}
scoring = 'accuracy'
for (model_name, model), params in adapt_models.items():
  kfold = KFold(n_splits=num_of_kfold_splits, random_state=seed)
  clf = GridSearchCV(model(), params, cv=kfold, scoring='accuracy')
  clf.fit(x, y)
  results[model_name] = clf

### Problem Transformation

In [ ]:
def convertToParamsForProblemTransformation(base_models):
  parameters = []
  for (model_name, model), params in base_models.items():
    parameter = {}
    parameter["classifier"] = model()
    for param_name, param_value in params.items():
      paramter["classifier__"+param_name] = param_value
    parameters.append(parameter)

  return paramters

In [ ]:
scoring = 'accuracy'
parameters = convertToParamsForProblemTransformation(base_models)
for strategy_name, strategy in problem_transform.items():
  kfold = KFold(n_splits=num_of_kfold_splits, random_state=seed)
  clf = GridSearchCV(model, parameters, cv=kfold, scoring='accuracy')
  clf.fit(x, y)
  results[model_name] = clf

### Ensembles of Classifiers (Under reconstruction)

#### Adaptation approaches

In [ ]:
def convertToParamsForProblemTransformation(base_models):
  parameters = []
  for (model_name, model), params in base_models.items():
    parameter = {}
    parameter["classifier"] = model()
    for param_name, param_value in params.items():
      paramter["classifier__"+param_name] = param_value
    parameters.append(parameter)

  return paramters

In [ ]:
scoring = 'accuracy'
for emsemble_name, emsemble_method in emsemble_methods.items():
  for strategy_name, strategy in problem_transform.items():
    for base_model_name, base_model in base_models.items():
      kfold = KFold(n_splits=num_of_kfold_splits, random_state=seed)
      model = strategy(
              classifier = base_model(),
              require_dense = [False, True] # depens on the data is sparse or dense
      )
      cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
      results.append(cv_results)
      names.append(model_name)
      msg = "%s: %f (%f)" % (model_name, cv_results.mean(), cv_results.std())
      print(msg)
      classifier = MajorityVotingClassifier(
    clusterer = FixedLabelSpaceClusterer(clusters = [[1,3,4], [0, 2, 5]]),
    classifier = ClassifierChain(classifier=GaussianNB())
)
classifier.fit(X_train,y_train)
predictions = classifier.predict(X_test)

In [ ]:
classifier = MajorityVotingClassifier(
    clusterer = FixedLabelSpaceClusterer(clusters = [[1,3,4], [0, 2, 5]]),
    classifier = ClassifierChain(classifier=GaussianNB())
)
classifier.fit(X_train,y_train)


# Evaluation 

In [ ]:
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()